# Data Preprocessing Tools

## Importing the libraries

In [41]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [42]:
dataset = pd.read_csv('Data.csv')
X = dataset.iloc[:, :-1].values #location of column by indices,
# la parentesi [] include il primo indice, ma esclude il secondo indice
y = dataset.iloc[:, -1].values

In [43]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


In [44]:
print(y)

['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']


## Taking care of missing data

In [45]:
# si può eliminare oppure replace con la media della colonna, o il most frequent.
from sklearn.impute import SimpleImputer
custom_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# fit method prenderà in considerazione i valori mancanti e li sostituirà con
# la strategia indicata
custom_imputer.fit(X[:, 1:3]) # parte da 0!!!
X[:, 1:3] = custom_imputer.transform(X[:, 1:3])


In [46]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


## Encoding categorical data

### Encoding the Independent Variable

In [47]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# non ci sono ordini numerici nella categoria country, per cui:
# [one hot enccoding] -> codifica a caldo: trasformarle in 3 colonne
custom_column_transformer = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
# quelle da modificare (transf) e quelle da lasciare (rem)
X = np.array(custom_column_transformer.fit_transform(X))
#abbiamo bisogno di forzare l'output come un NumPy Array

In [48]:
print(X)

[[1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


### Encoding the Dependent Variable

In [49]:
# mentre yes and no possono essere sostituiti da 0 e 1 (Label)
# molto più semplice gestire le due classi => binary outcomes
from sklearn.preprocessing import LabelEncoder

custom_label_encoder = LabelEncoder()
y = custom_label_encoder.fit_transform(y)

In [50]:
print(y)

[0 1 0 0 1 1 0 1 0 1]


## Splitting the dataset into the Training set and Test set

In [51]:
# importante scegliere COME e QUANDO dividerlo
# come si divide? c'è un metodo migliore degli altri
# meglio PRIMA splittare e DOPO fare feature scaling dopo aver splittato,
# in questo modo
# gli elementi che influenzano la media, il massimo, il minimo restano nell'insieme.
# il test set non deve influenzare in nessum modo il training set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
# questa funzione ci restituirà 4 matrici: una coppia di training l'altra di test
# l'ultimo parametro è il seed del random

In [52]:
print(X_train)

[[0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 35.0 58000.0]]


In [53]:
print(X_test)

[[0.0 1.0 0.0 30.0 54000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


In [54]:
print(y_train)

[0 1 0 0 1 1 0 1]


In [55]:
print(y_test)

[0 1]


## Feature Scaling

In [56]:
# starting from the theory: standardisation vs normalization
# normalisation funziona quando la distribuzione è normale, casi particolari
# standardisation funziona sempre

from sklearn.preprocessing import StandardScaler

custom_scaler = StandardScaler()
#La standard. dobbiamo applicarla alle variabili dummy? trasformate in 0 e 1? NO
# rischiamo di perdere la mappatura verso la variabile originale

#attenzione agli indici, abbiamo 3 colonno di dummy
# fit otterrà la media e la sd
# transform applicherà i cambi
X_train[:, 3:] = custom_scaler.fit_transform(X_train[:, 3:])
# applichiamo lo stesso scaler che abbiamo applicato all'insieme di test
# poichè la logica è stata calcolata su quei parametri.
# per questo facciamo solo TRANSFORM e non FIT
X_test[:, 3:] = custom_scaler.transform(X_test[:, 3:])

In [58]:
print(X_train)

[[0.0 0.0 1.0 -0.19159184384578545 -1.0781259408412425]
 [0.0 1.0 0.0 -0.014117293757057777 -0.07013167641635372]
 [1.0 0.0 0.0 0.566708506533324 0.633562432710455]
 [0.0 0.0 1.0 -0.30453019390224867 -0.30786617274297867]
 [0.0 0.0 1.0 -1.9018011447007988 -1.420463615551582]
 [1.0 0.0 0.0 1.1475343068237058 1.232653363453549]
 [0.0 1.0 0.0 1.4379472069688968 1.5749910381638885]
 [1.0 0.0 0.0 -0.7401495441200351 -0.5646194287757332]]


In [59]:
print(X_test)

[[0.0 1.0 0.0 -1.4661817944830124 -0.9069571034860727]
 [1.0 0.0 0.0 -0.44973664397484414 0.2056403393225306]]
